In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Lipinski, Fragments
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import DataPreprocessing as dprep
from sklearn.impute import SimpleImputer

In [7]:
training_data_path = 'training_smiles.csv'
test_data_path = 'test_smiles.csv'

training_data = pd.read_csv(training_data_path, dtype = {'ACTIVE': int})
test_data = pd.read_csv(test_data_path)

In [8]:
def extract_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)

    features = {}

    # Try nBits 2048, 1024, 512, 256
    # Morgan Fingerprint
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=512)
    for i in range(512):
        features[f'fp_{i}'] = morgan_fp[i]

    return features


    
training_features_df = training_data['SMILES'].apply(extract_fingerprints)

training_features_df = training_features_df.apply(pd.Series)

training_data_fingerprint = training_data.join(training_features_df)

training_data_fingerprint.to_csv('training_data_fingerprint.csv', index=False)


[11:47:46] WARNING: not removing hydrogen atom without neighbors


In [9]:

def extract_features(smiles):

    mol = Chem.MolFromSmiles(smiles)

    features = {}

    # Basic Properties
    features['num_atoms'] = mol.GetNumAtoms()
    features['num_bonds'] = mol.GetNumBonds()
    features['num_rings'] = mol.GetRingInfo().NumRings()

    # Molecular Descriptors
    for desc_name, desc_func in Descriptors.descList:
        features[desc_name] = desc_func(mol)

    # Lipinski Descriptors
    features['num_rotatable_bonds'] = Lipinski.NumRotatableBonds(mol)
    features['num_aromatic_rings'] = Lipinski.NumAromaticRings(mol)
    features['num_heteroatoms'] = Lipinski.NumHeteroatoms(mol)
    features['num_heavy_atoms'] = Lipinski.HeavyAtomCount(mol)
    features['num_h_donors'] = Lipinski.NumHDonors(mol)
    features['num_h_acceptors'] = Lipinski.NumHAcceptors(mol)
    features['num_aliphatic_rings'] = Lipinski.NumAliphaticRings(mol)
    features['num_saturated_rings'] = Lipinski.NumSaturatedRings(mol)
    features['num_aromatic_heterocycles'] = Lipinski.NumAromaticHeterocycles(mol)
    features['num_aromatic_carbocycles'] = Lipinski.NumAromaticCarbocycles(mol)
    features['num_aliphatic_heterocycles'] = Lipinski.NumAliphaticHeterocycles(mol)
    features['num_aliphatic_carbocycles'] = Lipinski.NumAliphaticCarbocycles(mol)

    # Fragment Descriptors
    for frag_func in dir(Fragments):
        if frag_func.startswith('fr_'):
            features[frag_func] = getattr(Fragments, frag_func)(mol)


    return features

training_features_df = training_data['SMILES'].apply(extract_features)

training_features_df = training_features_df.apply(pd.Series)

training_data_features = training_data.join(training_features_df)

training_data_features.to_csv('training_data_features.csv', index=False)



[12:30:17] WARNING: not removing hydrogen atom without neighbors
[12:30:17] WARNING: not removing hydrogen atom without neighbors
[12:30:17] WARNING: not removing hydrogen atom without neighbors


In [3]:
# Load the datasets
fingerprint_df = pd.read_csv('training_data_fingerprint.csv')
other_features_df = pd.read_csv('training_data_207_features.csv')

# Combine datasets
combined_df = pd.concat([fingerprint_df, other_features_df], axis=1)

#combined_df, column_filter = dprep.create_column_filter(combined_df)
#combined_df, imputation = dprep.create_imputation(combined_df)


# Select only numeric columns for imputation
numeric_cols = combined_df.select_dtypes(include=[np.number]).columns
numeric_df = combined_df[numeric_cols]

simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
numeric_df_imputed = pd.DataFrame(simple_imputer.fit_transform(numeric_df), columns=numeric_df.columns)


training_data = pd.concat([combined_df.drop(columns=numeric_cols), numeric_df_imputed], axis=1)


X = training_data.drop(columns=["INDEX", "SMILES", "ACTIVE"])
y = training_data["ACTIVE"]


In [44]:
y = combined_df['ACTIVE'].iloc[:, 0]

In [5]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

rf_clf.fit(X, y)

importances = rf_clf.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_importance_df.to_csv('feature_importances.csv', index=False)

print(feature_importance_df.head())


               Feature  Importance
537       BCUT2D_MRLOW    0.009652
512  MaxAbsEStateIndex    0.008942
514  MinAbsEStateIndex    0.008766
515     MinEStateIndex    0.008634
513     MaxEStateIndex    0.008383


In [47]:
def test_model_on_folds(X, y, classifier, use_smote=False, print_results=False, number_of_folds=5):

    # Apply SMOTE if required
    if use_smote:
        smote = SMOTE()
        X, y = smote.fit_resample(X, y)

    # Perform n-fold cross-validation
    y_proba = cross_val_predict(classifier, X, y, cv=number_of_folds, method='predict_proba')
    y_pred = np.argmax(y_proba, axis=1)
    conf_matrix = confusion_matrix(y, y_pred)
    class_report = classification_report(y, y_pred)
    auc_roc = roc_auc_score(y, y_proba[:, 1])

    if print_results:
        print("Confusion Matrix:")
        print(conf_matrix)
        print("\nClassification Report:")
        print(class_report)
        print("\nAUC-ROC Score:", auc_roc)

    return conf_matrix, class_report, auc_roc


In [48]:

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

num_features_list = [10, 30, 50, 80, 100, len(X.columns)]
feature_importances = pd.read_csv('feature_importances.csv')

results_without_SMOTE = {}
results_with_SMOTE = {}

def evaluate_model(X, y, classifier, num_features_list, use_smote):
    results = {}
    all_feature_names = X.columns.tolist()

    for num_features in num_features_list:
        n_best_features = feature_importances['Feature'].head(num_features).tolist()
        valid_features = [f for f in n_best_features if f in all_feature_names]

        # Call test_model_on_folds with valid features
        X_subset = X[valid_features]
        conf_matrix, class_report, auc_roc = test_model_on_folds(X_subset, y, classifier, use_smote=use_smote, print_results=True)
        results[str(num_features)] = [auc_roc]

    return results





In [49]:
# Evaluate without SMOTE
results_without_SMOTE = evaluate_model(X, y, rf_clf, num_features_list, use_smote=False)

print("Results without SMOTE")
for key, value in results_without_SMOTE.items():
    print(key, ":", value)


Confusion Matrix:
[[151414     32]
 [  1742     42]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    151446
           1       0.57      0.02      0.05      1784

    accuracy                           0.99    153230
   macro avg       0.78      0.51      0.52    153230
weighted avg       0.98      0.99      0.98    153230


AUC-ROC Score: 0.7166547867940201
Confusion Matrix:
[[151426     20]
 [  1743     41]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    151446
           1       0.67      0.02      0.04      1784

    accuracy                           0.99    153230
   macro avg       0.83      0.51      0.52    153230
weighted avg       0.98      0.99      0.98    153230


AUC-ROC Score: 0.7661129132946143
Confusion Matrix:
[[151426     20]
 [  1743     41]]

Classification Report:
              precision    recall  f1-score  

In [50]:
# Evaluate with SMOTE
results_with_SMOTE = evaluate_model(X, y, rf_clf, num_features_list, use_smote=True)

print("\nResults with SMOTE")
for key, value in results_with_SMOTE.items():
    print(key, ":", value)

Confusion Matrix:
[[149605   1841]
 [  1138 150308]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    151446
           1       0.99      0.99      0.99    151446

    accuracy                           0.99    302892
   macro avg       0.99      0.99      0.99    302892
weighted avg       0.99      0.99      0.99    302892


AUC-ROC Score: 0.9993570208781507
Confusion Matrix:
[[150762    684]
 [   468 150978]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    151446
           1       1.00      1.00      1.00    151446

    accuracy                           1.00    302892
   macro avg       1.00      1.00      1.00    302892
weighted avg       1.00      1.00      1.00    302892


AUC-ROC Score: 0.9998507236534854
Confusion Matrix:
[[151051    395]
 [   298 151148]]

Classification Report:
              precision    recall  f1-score  